## a JSON format compatible with the Pandas data structure
### introduction
The data type is not explicitely taken into account in the current JSON interface.     
    
The existing solution is to use a data schema in addition.

### proposal
To have a simple and compact solution, I propose to use the [JSON-NTV format (Named and Typed Value)](https://github.com/loco-philippe/NTV#readme) - which integrates the notion of type - and its JSON-TAB variation for tabular data.

### content
This NoteBook uses examples to present some key points

In [1]:
from json_ntv import NtvSingle, NtvList, Ntv, NtvConnector, NtvType, Namespace
from datetime import date, datetime
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(compact=True, width=100)
pr = pp.pprint
# the Series Connector is associated with NtvType 'field' in dicobj 
# the DataFrame Connector is associated with NtvType 'tab' in dicobj
option = {'encode_format': 'obj', 'dicobj': {'field': 'SeriesConnec', 'tab': 'DataFrameConnec'}}

## Series

### Simple example (json type)

In [2]:
field_data = {'value': [1, 2, 3]}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype conform to Ntv type
print('Pandas object :')
print(sr, '\n')
print('Json representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

Pandas object :
0    1
1    2
2    3
Name: value, dtype: int64 

Json representation : 
     {":field": {"value": [1, 2, 3]}}

is Json translation reversible ?  True


### Typed example

In [3]:
field_data = {'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21']}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype conform to Ntv type
print('Pandas object :')
print(sr, '\n')
print('Json representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

Pandas object :
0   1964-01-01
1   1985-02-05
2   2022-01-21
Name: dates, dtype: datetime64[ns] 

Json representation : 
     {":field": {"dates::datetime": ["1964-01-01T00:00:00.000", "1985-02-05T00:00:00.000", "2022-01-21T00:00:00.000"]}}

is Json translation reversible ?  True


### Examples with a non-Pandas type

In [4]:
field_data = {'dates::date': ['1964-01-01', '1985-02-05', '2022-01-21']}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype : object
print('Pandas object :')
print(sr, '\n')
print('Json representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

Pandas object :
0    1964-01-01
1    1985-02-05
2    2022-01-21
Name: dates::date, dtype: object 

Json representation : 
     {":field": {"dates::date": ["1964-01-01", "1985-02-05", "2022-01-21"]}}

is Json translation reversible ?  True


In [5]:
field_data = {'coord::point':    [[1,2], [3,4], [5,6]]}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype : object
print('Pandas object :')
print(sr, '\n')
print('Json representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

Pandas object :
0    POINT (1 2)
1    POINT (3 4)
2    POINT (5 6)
Name: coord::point, dtype: object 

Json representation : 
     {":field": {"coord::point": [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]}}

is Json translation reversible ?  True


C:\Opensource\anaconda3\envs\observ_v10_20230430\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)


### Categorical example

In [21]:
field_data = {"integer": [[1, 2], [0, 1, 0, 1]]}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype : object
print('Pandas object :')
print(sr, '\n')
print('Json representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

Pandas object :
0    1
1    2
2    1
3    2
Name: integer, dtype: category
Categories (2, int64): [1, 2] 

Json representation : 
     {":field": {"integer": [[1, 2], [0, 1, 0, 1]]}}

is Json translation reversible ?  True


In [6]:
field_data = {'dates': [{'::date': ['1964-01-01', '1985-02-05', '2022-01-21']}, [0, 1, 0, 2]]}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype : object
print('Pandas object :')
print(sr, '\n')
print('Json representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

Pandas object :
0    1964-01-01
1    1985-02-05
2    1964-01-01
3    2022-01-21
Name: dates::date, dtype: category
Categories (3, object): [1964-01-01, 1985-02-05, 2022-01-21] 

Json representation : 
     {":field": {"dates": [{"::date": ["1964-01-01", "1985-02-05", "2022-01-21"]}, [0, 1, 0, 2]]}}

is Json translation reversible ?  True


## DataFrame

### Simple example (json type)

In [8]:
tab_data = {'index':           [100, 200, 300, 400, 500, 600],
            'dates::date':     ['1964-01-01', '1985-02-05', '2022-01-21', '1964-01-01', '1985-02-05', '2022-01-21'], 
            'value':           [10, 10, 20, 20, 30, 30],
            'value32::int32':  [12, 12, 22, 22, 32, 32],
            'res':             [10, 20, 30, 10, 20, 30],
            'coord::point':    [[1,2], [3,4], [5,6], [7,8], [3,4], [5,6]],
            'names::string':   ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
            'unique:boolean':  True }
tab = Ntv.obj({':tab'  : tab_data})
df = tab.to_obj(**option)

print('Pandas object :')
print(df.dtypes, '\n')
print(df, '\n')
print('Json representation : \n    ', Ntv.obj(df))
print('\nis Json translation reversible ? ', df.equals(Ntv.obj(df).to_obj(**option)))

Pandas object :
dates::date     object
value            int64
value32          int32
res              int64
coord::point    object
names           string
unique            bool
dtype: object 

    dates::date  value  value32  res coord::point   names  unique
100  1964-01-01     10       12   10  POINT (1 2)    john    True
200  1985-02-05     10       12   20  POINT (3 4)    eric    True
300  2022-01-21     20       22   30  POINT (5 6)  judith    True
400  1964-01-01     20       22   10  POINT (7 8)    mila    True
500  1985-02-05     30       32   20  POINT (3 4)  hector    True
600  2022-01-21     30       32   30  POINT (5 6)   maria    True 

Json representation : 
     {":tab": {"index": [100, 200, 300, 400, 500, 600], "dates::date": ["1964-01-01", "1985-02-05", "2022-01-21", "1964-01-01", "1985-02-05", "2022-01-21"], "value": [10, 10, 20, 20, 30, 30], "value32::int32": [12, 12, 22, 22, 32, 32], "res": [10, 20, 30, 10, 20, 30], "coord::point": [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]

C:\Opensource\anaconda3\envs\observ_v10_20230430\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)
C:\Opensource\anaconda3\envs\observ_v10_20230430\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)


### Json data can be annotated

In [9]:
tab_data = {'index':           [100, 200, 300, 400, 500, 600],
            'dates::date':     ['1964-01-01', '1985-02-05', '2022-01-21', '1964-01-01', '1985-02-05', '2022-01-21'], 
            'value':           [10, 10, 20, 20, {'valid?': 30}, 30],
            'value32::int32':  [12, 12, 22, 22, 32, 32],
            'res':             {'res1': 10, 'res2': 20, 'res3': 30, 'res4': 10, 'res5': 20, 'res6': 30},
            'coord::point':    [[1,2], [3,4], [5,6], [7,8], {'same as 2nd point': [3,4]}, [5,6]],
            'names::string':   ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
            'unique:boolean':  True }
tab = Ntv.obj({':tab'  : tab_data})
df2 = tab.to_obj(**option)

print('\nis DataFrame identical ? ', df.equals(df2))


is DataFrame identical ?  True


C:\Opensource\anaconda3\envs\observ_v10_20230430\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)


In [10]:
from observation import Ilist

ntv2 = Ilist.ntv(Ntv.obj(df).val).to_ntv('default')
print(ntv2)
#print(Ilist.ntv(ntv2).to_ntv('full'))
df3 = Ntv.obj({':tab': Ilist.ntv(ntv2).to_ntv('full').to_obj()}).to_obj(encode_format='obj', dicobj={'tab': 'DataFrameConnec'})

print(df3.dtypes)
df3

{"index": [100, 200, 300, 400, 500, 600], "dates": [{"::date": ["1985-02-05", "2022-01-21", "1964-01-01"]}, [2, 0, 1, 2, 0, 1]], "value": [[30, 10, 20], [1, 1, 2, 2, 0, 0]], "value32": [{"::int32": [32, 12, 22]}, [1, 1, 2, 2, 0, 0]], "res": [[30, 10, 20], [1, 2, 0, 1, 2, 0]], "coord": [{"::point": [[5.0, 6.0], [7.0, 8.0], [1.0, 2.0], [3.0, 4.0]]}, [2, 3, 0, 1, 3, 0]], "names::string": ["john", "eric", "judith", "mila", "hector", "maria"], "unique": true}
dates::date     object
value            int64
value32          int32
res              int64
coord::point    object
names           string
unique            bool
dtype: object


C:\Opensource\anaconda3\envs\observ_v10_20230430\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)


,dates::date,value,value32,res,coord::point,names,unique
100,1964-01-01,10,12,10,POINT (1 2),john,True
200,1985-02-05,10,12,20,POINT (3 4),eric,True
300,2022-01-21,20,22,30,POINT (5 6),judith,True
400,1964-01-01,20,22,10,POINT (7 8),mila,True
500,1985-02-05,30,32,20,POINT (3 4),hector,True
600,2022-01-21,30,32,30,POINT (5 6),maria,True


In [11]:
from observation import Ilist

ntv2 = Ilist.ntv(Ntv.obj(df).val).to_ntv()
print(ntv2)
#print(Ilist.ntv(ntv2).to_ntv('full'))
df3 = Ntv.obj({':tab': Ilist.ntv(ntv2).to_ntv('full').to_obj()}).to_obj(encode_format='obj', dicobj={'tab': 'DataFrameConnec'})

print(df3.dtypes)
df3

{"index": [100, 200, 300, 400, 500, 600], "dates": [{"::date": ["1985-02-05", "2022-01-21", "1964-01-01"]}, 5, [1, 2, 2, 0]], "value": [[30, 10, 20], [1, 1, 2, 2, 0, 0]], "value32": [{"::int32": [32, 12, 22]}, 2], "res": [[20, 30, 10], 1], "coord": [{"::point": [[5.0, 6.0], [7.0, 8.0], [1.0, 2.0], [3.0, 4.0]]}, [2, 3, 0, 1, 3, 0]], "names::string": ["john", "eric", "judith", "mila", "hector", "maria"], "unique": true}
dates::date     object
value            int64
value32          int32
res              int64
coord::point    object
names           string
unique            bool
dtype: object


C:\Opensource\anaconda3\envs\observ_v10_20230430\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)


,dates::date,value,value32,res,coord::point,names,unique
100,1964-01-01,10,12,10,POINT (1 2),john,True
200,1985-02-05,10,12,20,POINT (3 4),eric,True
300,2022-01-21,20,22,30,POINT (5 6),judith,True
400,1964-01-01,20,22,10,POINT (7 8),mila,True
500,1985-02-05,30,32,20,POINT (3 4),hector,True
600,2022-01-21,30,32,30,POINT (5 6),maria,True


In [12]:
data   = {'index':           [10, 20, 30],
              'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21'], 
              'value':           [10, 20, {'valid?': 30}],
              'value32::int32':  [10, 20, 30],
              'res':             {'res1': 10, 'res2': 20, 'res3': 30},
              'coord::point':    [[1,2], [3,4], [5,6]],
              'names::string':   ['john', 'eric', 'judith'],
              'unique:boolean': True}
df  = Ntv.obj({':tab'  : data}).to_obj  (encode_format='obj', dicobj={'tab': 'DataFrameConnec'})
print(df.dtypes)
df


dates           datetime64[ns]
value                    int64
value32                  int32
res                      int64
coord::point            object
names                   string
unique                    bool
dtype: object


C:\Opensource\anaconda3\envs\observ_v10_20230430\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)


,dates,value,value32,res,coord::point,names,unique
10,1964-01-01,10,10,10,POINT (1 2),john,True
20,1985-02-05,20,20,20,POINT (3 4),eric,True
30,2022-01-21,30,30,30,POINT (5 6),judith,True


## Categorical

### Series

In [13]:
s_obj = pd.Series(["a", "b", "c", "a"], dtype="category")
i_obj = pd.Series([1,2,1,2], dtype="category")

print('categorical object Series  : ', Ntv.obj(s_obj))
print('categorical integer Series : ', Ntv.obj(i_obj))

categorical object Series  :  {":field": [["a", "b", "c"], [0, 1, 2, 0]]}
categorical integer Series :  {":field": [[1, 2], [0, 1, 0, 1]]}


In [14]:
print(s_obj.equals(Ntv.obj(s_obj).to_obj(**option)))
print(i_obj.equals(Ntv.obj(i_obj).to_obj(**option)))

True
True


In [15]:
s_obj = pd.Series(["a", "b", "c", "a"], name='string', dtype="category")
i_obj = pd.Series([1,2,1,2], name='integer', dtype="category")
o_obj = pd.Series(['1964-01-01', '1985-02-05', '1964-01-01', '2022-01-21'], name='dates::date', dtype="category")

print('categorical object Series  : ', Ntv.obj(s_obj))
print('categorical integer Series : ', Ntv.obj(i_obj))
print('categorical object Series : ', Ntv.obj(o_obj))                 
                   

categorical object Series  :  {":field": {"string": [["a", "b", "c"], [0, 1, 2, 0]]}}
categorical integer Series :  {":field": {"integer": [[1, 2], [0, 1, 0, 1]]}}
categorical object Series :  {":field": {"dates": [["1964-01-01", "1985-02-05", "2022-01-21"], [0, 1, 0, 2]]}}


In [16]:
print(s_obj.equals(Ntv.obj(s_obj).to_obj(**option)))
print(i_obj.equals(Ntv.obj(i_obj).to_obj(**option)))
print(o_obj.equals(Ntv.obj(o_obj).to_obj(**option)))

True
True
True


In [17]:
sr2 = Ntv.obj(o_obj).to_obj(**option)
print(type(sr2.cat.categories[0]))
print(o_obj)
print(sr2)

<class 'str'>
0    1964-01-01
1    1985-02-05
2    1964-01-01
3    2022-01-21
Name: dates::date, dtype: category
Categories (3, object): ['1964-01-01', '1985-02-05', '2022-01-21']
0    1964-01-01
1    1985-02-05
2    1964-01-01
3    2022-01-21
Name: dates, dtype: category
Categories (3, object): ['1964-01-01', '1985-02-05', '2022-01-21']


### Dataframe

In [18]:
s_df = pd.DataFrame({"A": list("abca"), "B": list("bccd")}, dtype="category")

print('categorical object DataFrame  : ')
print(Ntv.obj(s_df))

categorical object DataFrame  : 
{":tab": {"index": [0, 1, 2, 3], "A": [["a", "b", "c"], [0, 1, 2, 0]], "B": [["b", "c", "d"], [0, 1, 1, 2]]}}


In [19]:
print(s_df.equals(Ntv.obj(s_df).to_obj(**option)))
print(Ntv.obj(s_df).to_obj(**option))

True
   A  B
0  a  b
1  b  c
2  c  c
3  a  d
